# Desafio 2 - Regressão: Treinamento de Modelo

## Carregamento de features em diferentes formatos e target

In [2]:
import pickle
file_path = '/content/drive/MyDrive/curso_machine-learning-python/datasets/proccessed_insurance_data.pkl'
processed_insurance_data = open(file_path, 'rb')
target = pickle.load(processed_insurance_data)
all_features = pickle.load(processed_insurance_data)
scaled_all_features = pickle.load(processed_insurance_data)
high_corr_features = pickle.load(processed_insurance_data)
scaled_high_corr_features = pickle.load(processed_insurance_data)
processed_insurance_data.close()

In [3]:
import numpy as np
import pandas as pd

## Verificação de dois dos conjuntos carregados

In [4]:
target_df = pd.DataFrame(target)
target_df.shape

(1338, 1)

In [5]:
all_features_df = pd.DataFrame(all_features)
all_features_df.shape

(1338, 11)

## Separação de dados em treino e teste

In [6]:
def divide_dataset_treino_teste(source_features, source_target):
  from sklearn.model_selection import train_test_split
  features_treino, features_teste, target_treino, target_teste = train_test_split(
    source_features, source_target, test_size = 0.3, random_state = 0
  )
  return features_treino, features_teste, target_treino, target_teste

In [198]:
features_treino, features_teste, target_treino, target_teste = divide_dataset_treino_teste(all_features, target)

## Treinamento de modelos

### Multiple Linear Regression

In [8]:
from sklearn.linear_model import LinearRegression

In [9]:
lin_reg_model = LinearRegression()

In [10]:
lin_reg_model.fit(features_treino, target_treino)

LinearRegression()

In [11]:
lin_reg_model.score(features_treino, target_treino)

0.7309569871174701

In [12]:
lin_reg_model.score(features_teste, target_teste)

0.7909160991789904

#### Resultados

**Modelo:** Regressão Linear Múltipla

---

**Conjunto de features:** all_features

**score em treino:** 0.73

**score em teste:** 0.79

---

**Conjunto de features:** scaled_all_features

**score em treino:** 0.73

**score em teste:** 0.79

---


**Conjunto de features:** high_corr_features

**score em treino:** 0.70

**score em teste:** 0.76

---


**Conjunto de features:** scaled_high_corr_features

**score em treino:** 0.70

**score em teste:** 0.76



As features possuem muitas dimensões, não vai ser possível fazer um gráfico com a reta de regressão.

#### Métricas de Desempenho

In [13]:
from sklearn.metrics import root_mean_squared_error

In [14]:
predicted_target = lin_reg_model.predict(features_teste)
print(f"Erro médio usando all_features: {root_mean_squared_error(target_teste, predicted_target)}")

Erro médio usando all_features: 5774.296305780868


#### Validação Cruzada

In [15]:
from sklearn.model_selection import KFold, cross_val_score

In [16]:
k_fold = KFold(n_splits = 10, shuffle = True, random_state = 0)

In [17]:
modelo = LinearRegression()

In [18]:
resultado = cross_val_score(modelo, all_features, target, cv = k_fold)

In [19]:
print(f"Coeficiente R² médio: {resultado.mean()}")
print(f"Coeficiente R² inferior: {resultado.mean() - resultado.std()}")
print(f"Coeficiente R² superior: {resultado.mean() + resultado.std()}")

Coeficiente R² médio: 0.7356373982088413
Coeficiente R² inferior: 0.680117796174853
Coeficiente R² superior: 0.7911570002428296


#### Conclusão

Modelo baseado em Regressão Linear Múltipla conseguiu se ajustar em 79% à target. É um dos modelos mais simples, então parece um bom resultado. Vamos testar outros modelos em seguida.

### Support Vector Regressor

In [20]:
from sklearn.svm import SVR

In [21]:
# default kernel is rbf
sup_vec_regressor = SVR(kernel = 'rbf')

In [22]:
sup_vec_regressor.fit(features_treino, target_treino)

SVR()

In [23]:
sup_vec_regressor.score(features_treino, target_treino)

-0.09007465037715123

In [24]:
sup_vec_regressor.score(features_teste, target_teste)

-0.0849999771017036

Do que o Support Vector Regressor precisa mesmo? Padronizar escalas de variáveis, tanto dependentes quando independentes.

In [25]:
from sklearn.preprocessing import StandardScaler
feature_scaler = StandardScaler()
scaled_features_treino = feature_scaler.fit_transform(features_treino)

In [26]:
target_scaler = StandardScaler()
scaled_target_treino = target_scaler.fit_transform(target_treino.reshape(-1, 1))

In [27]:
SVR_wscaled = SVR(kernel = 'rbf')
SVR_wscaled.fit(scaled_features_treino, scaled_target_treino)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR()

In [28]:
SVR_wscaled.score(scaled_features_treino, scaled_target_treino)

0.8374931487203509

In [29]:
scaled_features_teste = feature_scaler.fit_transform(features_teste)

In [30]:
scaled_target_teste = target_scaler.fit_transform(target_teste.reshape(-1,1))

In [31]:
SVR_wscaled.score(scaled_features_teste, scaled_target_teste)

0.8752604597810798

O coeficiente de determinação $R^2$ ficou melhor no SVR.

#### Resultados

**Modelo:** Support Vector Regressor

---

**Conjunto de features:** scaled_all_features and scaled_target

**score em treino:** 0.84

**score em teste:** 0.88


#### Métricas de Desempenho

In [32]:
from sklearn.metrics import root_mean_squared_error

In [33]:
predicted_target = SVR_wscaled.predict(scaled_features_teste)

In [34]:
target_teste_inversed = target_scaler.inverse_transform(scaled_target_teste)

In [35]:
predicted_target_inversed = target_scaler.inverse_transform(predicted_target.reshape(-1, 1))

In [36]:
root_mean_squared_error(target_teste_inversed, predicted_target_inversed)

4460.061407282233

#### Validação Cruzada

In [37]:
from sklearn.model_selection import KFold, cross_val_score

In [38]:
X_scaler = StandardScaler()

In [39]:
y_scaler = StandardScaler()

In [40]:
X_scaled = X_scaler.fit_transform(all_features)

In [41]:
y_scaled = y_scaler.fit_transform(target.reshape(-1, 1))

In [42]:
modelo = SVR(kernel = 'rbf')

In [43]:
k_fold = KFold(n_splits = 10, shuffle = True, random_state = 0)

In [44]:
resultado = cross_val_score(modelo, X_scaled, y_scaled.ravel(), cv = k_fold)

In [45]:
print(f"Coeficiente R² médio: {resultado.mean()}")
print(f"Coeficiente R² inferior: {resultado.mean() - resultado.std()}")
print(f"Coeficiente R² superior: {resultado.mean() + resultado.std()}")

Coeficiente R² médio: 0.8356454051983156
Coeficiente R² inferior: 0.7880730379941618
Coeficiente R² superior: 0.8832177724024695


#### Conclusão

O regressor baseado em support vector machines se saiu melhor do que a regressão linear, tanto no coeficiente R² quanto na métrica de erro RMSE.
Entre os 2, deve ser escolhido SVR. Apesar disso, o contra do SVR é que exige várias transformações de escala para poder ser utilizado. Talvez por isso, muitas vezes as pessoas escolhem outros modelos.

### Decision Tree Regressor

In [58]:
from sklearn.tree import DecisionTreeRegressor

In [59]:
tree = DecisionTreeRegressor(max_depth = 4, criterion='poisson', random_state = 0)

In [60]:
tree.fit(features_treino, target_treino)

DecisionTreeRegressor(criterion='poisson', max_depth=4, random_state=0)

In [61]:
tree.score(features_treino, target_treino)

0.8531650151210386

In [62]:
tree.score(features_teste, target_teste)

0.8854782318253775

squared_error:
- treino: 0.852 ~ 0.85
- teste: 0.884 ~ 0.88

poisson:
- treino: 0.853 ~ 0.85
- teste: 0.885 ~ 0.89

#### Resultados

all_features == scaled_all_features

R² em treino: 0.85

R² em teste: 0.89

#### Métricas de Desempenho

In [63]:
from sklearn.metrics import root_mean_squared_error

In [64]:
predicted_target_teste = tree.predict(features_teste)
root_mean_squared_error(target_teste, predicted_target_teste)

4273.490975094296

#### Validação Cruzada

In [65]:
from sklearn.model_selection import KFold, cross_val_score

In [66]:
k_fold = KFold(n_splits = 10, shuffle = True, random_state = 0)

In [67]:
modelo = DecisionTreeRegressor(max_depth = 4, criterion='poisson', random_state = 0)

In [68]:
resultado = cross_val_score(modelo, all_features, target)

In [69]:
print(f"Coeficiente R² médio: {resultado.mean()}")
print(f"Coeficiente R² inferior: {resultado.mean() - resultado.std()}")
print(f"Coeficiente R² superior: {resultado.mean() + resultado.std()}")

Coeficiente R² médio: 0.8510140323464425
Coeficiente R² inferior: 0.8141620277296231
Coeficiente R² superior: 0.8878660369632618


#### Conclusão

O Regressor baseado em árvore de decisão superou o Suppor Vector Regressor. A diferença foi pequena, mas ele obteve melhores coeficiente R² além de menor erro com RSME. Além disso, ainda tem a vantagem de exigir menos pré/pós-processamento.

### Random Forest

In [193]:
from sklearn.ensemble import RandomForestRegressor

In [203]:
random_forest = RandomForestRegressor(n_estimators = 100, criterion = 'friedman_mse', max_depth = 6, random_state = 0)

In [204]:
random_forest.fit(features_treino, target_treino)

RandomForestRegressor(criterion='friedman_mse', max_depth=6, random_state=0)

In [205]:
random_forest.score(features_treino, target_treino)

0.9047018875232218

In [206]:
random_forest.score(features_teste, target_teste)

0.8798194991833832

#### Resultados

all_features == scaled_all_features

Random Forest

- score em treino: 0.90

- score em teste: 0.88

#### Métricas de Desempenho

In [207]:
from sklearn.metrics import root_mean_squared_error

In [208]:
predicted_target_teste = random_forest.predict(features_teste)
root_mean_squared_error(target_teste, predicted_target_teste)

4377.798554276961

In [210]:
# Observação: às vezes o algoritmo pode ter um bom score e ter um RMSE ruim. Mudei
# o n_estimators de 250 para 100 para diminuir esse erro. Me parece que estava tendendo ao
# overfitting.

#### Validação Cruzada

In [212]:
from sklearn.model_selection import KFold, cross_val_score

In [214]:
k_fold = KFold(n_splits = 10, shuffle = True, random_state = 0)

In [215]:
modelo = RandomForestRegressor(n_estimators = 100, criterion = 'friedman_mse', max_depth = 6, random_state = 0)

In [216]:
resultado = cross_val_score(modelo, all_features, target, cv = k_fold)

In [217]:
print(f"Coeficiente R² médio: {resultado.mean()}")
print(f"Coeficiente R² inferior: {resultado.mean() - resultado.std()}")
print(f"Coeficiente R² superior: {resultado.mean() + resultado.std()}")

Coeficiente R² médio: 0.8502056088233102
Coeficiente R² inferior: 0.807442464238317
Coeficiente R² superior: 0.8929687534083034


#### Conclusão

Random Forest ficou muito parecido com o Decision Tree e ambos foram melhores que o SVR.
Devido à simplicidade, acredito que escolheria o Decision Tree até o momento.
Nas próximas seções, vamos ver 3 dos algoritmos mais poderosos, XGBoost, LightGBM e CatBoost.
